In [1]:
import boto3

s3 = boto3.client("s3")
bucket_name = "sagemaker-us-east-1-877305355343"
s3.download_file(bucket_name, "videos/WIN_20250616_17_16_29_Pro.mp4", "WIN_20250616_17_16_29_Pro.mp4")

In [2]:
import cv2
import os

video_path = "WIN_20250616_17_16_29_Pro.mp4"
output_dir = "video_frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_rate = 1  # Extract 1 frame per second
frame_count = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % int(cap.get(cv2.CAP_PROP_FPS) * frame_rate) == 0:
        frame_filename = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        saved_count += 1
cap.release()

print(f"✓ {saved_count} frames extracted to '{output_dir}'")

✓ 19 frames extracted to 'video_frames'


In [3]:
import boto3
import os

s3 = boto3.client('s3')
bucket_name = 'sagemaker-us-east-1-877305355343'
s3_prefix = 'batch_input/'

for filename in os.listdir(output_dir):
    if filename.endswith(".jpg"):
        local_path = os.path.join(output_dir, filename)
        s3.upload_file(local_path, bucket_name, s3_prefix + filename)

print("✓ All frames uploaded to S3.")

✓ All frames uploaded to S3.
